In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from tqdm import tqdm


In [4]:
movie_metadata= pd.read_csv('movies_metadata.csv')

<ipython-input-4-cc10dbac3bbf>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_metadata= pd.read_csv('movies_metadata.csv')


In [5]:
movie_keywords= pd.read_csv('keywords.csv')

In [6]:
movie_keywords['keywords']

0        [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1        [{'id': 10090, 'name': 'board game'}, {'id': 1...
2        [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3        [{'id': 818, 'name': 'based on novel'}, {'id':...
4        [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords, Length: 46419, dtype: object

In [7]:
movie_credits=pd.read_csv('credits.csv')

In [8]:
movie_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [9]:
# Considering movies that have vote count>50
movie_metadata = movie_metadata[movie_metadata['vote_count']>=50]

In [10]:
movie_metadata = movie_metadata[['id','original_title','overview','genres']]

In [11]:
#  Duplicating the Title Column
movie_metadata['title'] = movie_metadata['original_title'].copy()

In [12]:
movie_metadata.reset_index(inplace=True, drop=True)
movie_metadata.head()

,id,original_title,overview,genres,title
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men
3,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II
4,949,Heat,"Obsessive master thief, Neil McCauley leads a ...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat


In [13]:
movie_keywords.head()


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [14]:
movie_credits.head()


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [15]:
movie_credits = movie_credits[['id','cast']]

In [16]:
movie_metadata = movie_metadata[movie_metadata['id'].str.isnumeric()]

In [22]:

movie_metadata['id'] = movie_metadata['id'].astype(int)
#Merging movie_metadata and movie_keywords 
dff = pd.merge(movie_metadata, movie_keywords, on='id', how='left')

dff.reset_index(inplace=True, drop=True)

In [23]:
# Merge with movie credits
dff = pd.merge(dff, movie_credits, on='id', how='left')

dff.reset_index(inplace=True, drop=True)

**Data Cleaning & Feature Extraction**

In [24]:
# Preprocessing genres, keywords, cast columns to extract keywords, genre and cast details
dff['genres'] = dff['genres'].apply(lambda x: [i['name'] for i in eval(x)])
dff['genres'] = dff['genres'].apply(lambda x: ' '.join([i.replace(" ","") for i in x]))
dff['keywords'].fillna('[]', inplace=True)
dff['keywords'] = dff['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
dff['keywords'] = dff['keywords'].apply(lambda x: ' '.join([i.replace(" ",'') for i in x]))
dff['cast'].fillna('[]', inplace=True)
dff['cast'] = dff['cast'].apply(lambda x: [i['name'] for i in eval(x)])
dff['cast'] = dff['cast'].apply(lambda x: ' '.join([i.replace(" ",'') for i in x]))


In [26]:
dff['key_tags'] = dff['overview'] + ' ' + dff['genres'] +  ' ' + dff['original_title'] + ' ' + dff['keywords'] + ' ' + dff['cast']

In [27]:
dff

,id,original_title,overview,genres,title,keywords,cast,tags,key_tags
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family,Toy Story,jealousy toy boy friendship friends rivalry bo...,TomHanks TimAllen DonRickles JimVarney Wallace...,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,Adventure Fantasy Family,Jumanji,boardgame disappearance basedonchildren'sbook ...,RobinWilliams JonathanHyde KirstenDunst Bradle...,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,Romance Comedy,Grumpier Old Men,fishing bestfriend duringcreditsstinger oldmen,WalterMatthau JackLemmon Ann-Margret SophiaLor...,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,Father of the Bride Part II,baby midlifecrisis confidence aging daughter m...,SteveMartin DianeKeaton MartinShort KimberlyWi...,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...
4,949,Heat,"Obsessive master thief, Neil McCauley leads a ...",Action Crime Drama Thriller,Heat,robbery detective bank obsession chase shootin...,AlPacino RobertDeNiro ValKilmer JonVoight TomS...,"Obsessive master thief, Neil McCauley leads a ...","Obsessive master thief, Neil McCauley leads a ..."
...,...,...,...,...,...,...,...,...,...
9292,430365,À bras ouverts,Jean-Étienne Fougerole is an intellectual bohe...,Comedy,À bras ouverts,,ChristianClavier AryAbittan ElsaZylberstein Cy...,Jean-Étienne Fougerole is an intellectual bohe...,Jean-Étienne Fougerole is an intellectual bohe...
9293,248705,Les Visiteurs: La Révolution,"Stuck in the corridors of time, Godefroy de Mo...",Comedy,Les Visiteurs: La Révolution,nazis castle timetravel robespierre,JeanReno ChristianClavier FranckDubosc KarinVi...,"Stuck in the corridors of time, Godefroy de Mo...","Stuck in the corridors of time, Godefroy de Mo..."
9294,44918,Titanic II,On the 100th anniversary of the original voyag...,Action Adventure Thriller,Titanic II,suspense,ShaneVanDyke MarieWestbrook BruceDavison Brook...,On the 100th anniversary of the original voyag...,On the 100th anniversary of the original voyag...
9295,455661,In a Heartbeat,A closeted boy runs the risk of being outed by...,Family Animation Romance Comedy,In a Heartbeat,love teenager lgbt short,,A closeted boy runs the risk of being outed by...,A closeted boy runs the risk of being outed by...


In [28]:
dff.drop(columns=['genres','overview','original_title','keywords','cast'], inplace=True)

In [29]:
dff.head()

,id,title,tags,key_tags
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...
4,949,Heat,"Obsessive master thief, Neil McCauley leads a ...","Obsessive master thief, Neil McCauley leads a ..."


In [30]:
dff.drop(dff[dff['key_tags'].isnull()].index, inplace=True)

In [31]:
dff.drop_duplicates(inplace=True)

**Converting Text to Numbers using TF-IDF**

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)

transformed_data = tfidf.fit_transform(dff['key_tags'].values)
transformed_dataframe = pd.DataFrame(transformed_data.toarray(), index=dff['key_tags'].index.tolist())


In [42]:
transformed_dataframe

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Explored TruncatedSVD which is also a dimensionality reduction technique. This Algorithm can better handle outliers and non-transformed data compared  other dimensionality reduction techniques like PCA

In [43]:

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=3000)

reduced_data = svd.fit_transform(transformed_dataframe)



In [44]:
svd.explained_variance_ratio_.cumsum()

array([0.00550886, 0.01399735, 0.02099312, ..., 0.99999893, 1.        ,
       1.        ])

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(reduced_data)


Function to recommend movies to a user based on the movie the user has watched

In [46]:
def recommend_movie(title):
    movie_id = dff[dff['title']==title].index[0]
    distances = similarity[movie_id]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:5]
    
    for i in movie_list:
        print(dff.iloc[i[0]].title)

In [47]:
recommend_movie('The Matrix')


The Matrix Revolutions
The Matrix Revisited
The Matrix Reloaded
The Animatrix


### **Collaborative Filtering**

In [49]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095460 sha256=1eaecf867cca6ed8b713b7423c64b8ac692490e340cf8e89e40e7713ecf70608
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [50]:
from surprise import Dataset, Reader

from surprise.prediction_algorithms.matrix_factorization import SVD

from surprise import accuracy

In [52]:
import pandas as pd
ratings_data = pd.read_csv("ratings_small.csv")

movie_metadata = pd.read_csv("movies_metadata.csv")

ratings_data.head()

<ipython-input-52-3f249cdc73ca>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_metadata = pd.read_csv("movies_metadata.csv")


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [53]:
movie_metadata = movie_metadata[movie_metadata['vote_count']>50][['id','title']]

movie_ids = [int(i) for i in movie_metadata['id'].values]

ratings_data = ratings_data[ratings_data['movieId'].isin(movie_ids)]

ratings_data.reset_index(inplace=True, drop=True)


In [55]:

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,5), skip_lines=1)

df = Dataset.load_from_df(ratings_data[['userId','movieId','rating']], reader=reader)

training_data = df.build_full_trainset()
svd = SVD()
svd.fit(training_data)


In [56]:
svd.predict(uid=3,iid=2959,r_ui=5.0)

Prediction(uid=3, iid=2959, r_ui=5.0, est=4.197563594322671, details={'was_impossible': False})

In [67]:
def recommend_movie(data, movie_metadata, user_id, number_of_movies, algorithm):
    recommended_list = []
    interactions_matrix = data.pivot(index='userId', columns='movieId', values='rating')
    non_interacted_movies = interactions_matrix.loc[user_id][interactions_matrix.loc[user_id].isnull()].index.tolist()
    for item_id in non_interacted_movies:
        prediction = algorithm.predict(user_id, item_id).est
        
        movie_name = movie_metadata[movie_metadata['id']==str(item_id)]['title'].values[0]
        recommended_list.append((movie_name, prediction))

    recommended_list.sort(key=lambda x: x[1], reverse=True)

    return recommended_list[:number_of_movies] 

In [68]:
recommend_movie(data=ratings_data,movie_metadata=movie_metadata, user_id=654, number_of_movies=10, algorithm=svd)

[('The Thomas Crown Affair', 4.994751331307456),
 ('Dead Man', 4.969477043534001),
 ('The Sixth Sense', 4.959233135054396),
 ('Nell', 4.952224585254854),
 ('While You Were Sleeping', 4.944744479722165),
 ('Frankenstein', 4.940587807896221),
 ("Don't Worry, I'm Fine", 4.92196900673639),
 ('The Thomas Crown Affair', 4.899751530889703),
 ('Birdman of Alcatraz', 4.889609145290421),
 ('Madagascar', 4.853461852935395)]

**Memory based Recommender System**



**User Based Collaborative Field**

In [4]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095448 sha256=cbc920c178f01793b3a75ab148f014c4a7ed14a94c17ffc183765b0ff102d902
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [81]:
from surprise.prediction_algorithms.knns import KNNBasic

In [82]:
sim_options = {'name': 'cosine',
               'user_based': True}

similar_users = KNNBasic(sim_options=sim_options, verbose=False, random_state=33)

similar_users.fit(training_data)

In [ ]:
similar_users.predict(uid=3,iid=2959,r_ui=5.0)

In [74]:
recommend_movie(ratings_data, movie_metadata, 671,10,similar_users)

[('The Wizard', 5),
 ('Rio Bravo', 5),
 ('The Celebration', 5),
 ('Spider-Man 3', 5),
 ('A Streetcar Named Desire', 5),
 ('Gentlemen Prefer Blondes', 5),
 ('The Evil Dead', 5),
 ('JFK', 5),
 ("Singin' in the Rain", 5),
 ("Frank Herbert's Dune", 5)]

**Item Based Collaborative Filtering**

In [78]:
similar_options = {'name': 'cosine',
               'user_based': False}

similar_item = KNNBasic(sim_options=sim_options, verbose=False, random_state=33)

similar_item.fit(training_data)

In [ ]:
similar_item.predict(uid=3,iid=2959,r_ui=5.0)

In [80]:
recommend_movie(ratings_data, movie_metadata, 671,10,similar_item)

[('The Wizard', 5),
 ('Rio Bravo', 5),
 ('The Celebration', 5),
 ('Spider-Man 3', 5),
 ('A Streetcar Named Desire', 5),
 ('Gentlemen Prefer Blondes', 5),
 ('The Evil Dead', 5),
 ('JFK', 5),
 ("Singin' in the Rain", 5),
 ("Frank Herbert's Dune", 5)]